In [1]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")
import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [2]:
num_rows_batch_publish=5000 # number of objects to be batch published to Scorpio

# Notes:
Consider every hexagon as an entity, and sensor data as properties

In [3]:
import pandas as pd
import geopandas as gpd



In [4]:
# import building datasets
gdf_madrid = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-ESP_8_1_3_10_1-Madrid.gpkg')
gdf_berlin = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-DEU_3_1_1_1_1-Berlin.gpkg')

gdf_madrid= gdf_madrid.rename(columns={"id": "building_id"})
gdf_berlin=gdf_berlin.rename(columns={"id": "building_id"})

gdf_list = [gdf_madrid, gdf_berlin]

In [5]:
gdf_berlin

,building_id,height,age,type,id_source,type_source,geometry
0,v0.1-DEU.3.1.1.1_1-0,2.77,NaN,NaN,BLDG_00030000006a7af7,2862.0,"POLYGON ((4563917.779 3263885.727, 4563917.780..."
1,v0.1-DEU.3.1.1.1_1-1,2.38,NaN,NaN,BLDG_00030000006a7ac7,2862.0,"POLYGON ((4563910.579 3263877.229, 4563910.578..."
2,v0.1-DEU.3.1.1.1_1-2,5.43,NaN,NaN,BLDG_0003000f00190176,1301.0,"POLYGON ((4563908.722 3263914.760, 4563908.721..."
3,v0.1-DEU.3.1.1.1_1-3,4.27,NaN,NaN,BLDG_000300090023bdbd,1004.0,"POLYGON ((4563917.334 3263793.450, 4563917.333..."
4,v0.1-DEU.3.1.1.1_1-4,3.31,NaN,NaN,BLDG_0003000f00190236,1301.0,"POLYGON ((4563836.581 3263926.777, 4563836.580..."
...,...,...,...,...,...,...,...
538547,v0.1-DEU.3.1.1.1_1-538547,4.24,NaN,NaN,DEB_LOD2_UUID_a9194181-b0c2-4a06-bbab-9056106c...,NaN,"POLYGON ((4542939.600 3276469.358, 4542935.639..."
538548,v0.1-DEU.3.1.1.1_1-538548,6.52,NaN,NaN,DEB_LOD2_UUID_ce085ade-285c-43ef-a3e6-e49e4e70...,NaN,"POLYGON ((4544272.142 3282876.577, 4544272.393..."
538549,v0.1-DEU.3.1.1.1_1-538549,4.15,NaN,NaN,BLDG_0003000a004c5bb9,1411.0,"POLYGON ((4546662.693 3277914.566, 4546662.693..."
538550,v0.1-DEU.3.1.1.1_1-538550,2.67,NaN,NaN,DEB_LOD2_UUID_5a5100f8-82fc-442e-bac0-88c7aa68...,NaN,"POLYGON ((4543757.047 3282169.972, 4543757.046..."


In [6]:
gdf_madrid

,building_id,height,age,type,id_source,type_source,geometry
0,v0.1-ESP.8.1.3.10_1-0,2.5,2000.0,residential,ES.SDGC.BU.50224A9VK2952S,1_residential,"POLYGON ((3147236.260 2050240.056, 3147239.537..."
1,v0.1-ESP.8.1.3.10_1-1,7.5,2000.0,residential,ES.SDGC.BU.50224A9VK2952S_part_1,1_residential,"POLYGON ((3147235.313 2050243.353, 3147233.145..."
2,v0.1-ESP.8.1.3.10_1-2,0.0,2002.0,residential,ES.SDGC.BU.50224B0VK2952S,1_residential,"POLYGON ((3147242.687 2050233.051, 3147241.021..."
3,v0.1-ESP.8.1.3.10_1-3,7.5,2002.0,residential,ES.SDGC.BU.50224B0VK2952S_part_1,1_residential,"POLYGON ((3147249.439 2050243.941, 3147246.028..."
4,v0.1-ESP.8.1.3.10_1-4,7.5,2000.0,residential,ES.SDGC.BU.50224B1VK2952S,1_residential,"POLYGON ((3147248.238 2050266.100, 3147256.448..."
...,...,...,...,...,...,...,...
155465,v0.1-ESP.8.1.3.10_1-155465,0.0,2008.0,residential,ES.SDGC.BU.9042412VK4984S_part_4,1_residential,"POLYGON ((3170930.744 2047829.963, 3170932.675..."
155466,v0.1-ESP.8.1.3.10_1-155466,7.5,2008.0,residential,ES.SDGC.BU.9042412VK4984S_part_5,1_residential,"POLYGON ((3170944.699 2047826.857, 3170946.682..."
155467,v0.1-ESP.8.1.3.10_1-155467,2.5,2008.0,residential,ES.SDGC.BU.9141402VK4994S,1_residential,"POLYGON ((3170975.309 2047700.803, 3170972.545..."
155468,v0.1-ESP.8.1.3.10_1-155468,7.5,2008.0,residential,ES.SDGC.BU.9141402VK4994S_part_1,1_residential,"POLYGON ((3170968.556 2047728.375, 3170971.381..."


In [7]:
# Import hexagonal grid:

madrid_hexagons_gdf = pd.read_pickle('./../cgarrido/madrid_data/hex_grid/hexagons.pkl')
berlin_hexagons_gdf=  pd.read_pickle('./../cgarrido/berlin_data/hex_grid/hexagons.pkl')
berlin_hexagons_gdf

,hexagon_id,hexagon_x,hexagon_y,geometry,lat,lon
0,0,0,0,"POLYGON ((1490671.563 6896601.483, 1490769.010...",52.525749,13.409063
1,1,1,1,"POLYGON ((1486732.502 6896205.302, 1486830.610...",52.523589,13.373677
2,2,0,1,"POLYGON ((1488848.594 6893262.031, 1488946.321...",52.507496,13.392682
3,3,1,0,"POLYGON ((1488555.212 6899545.594, 1488653.039...",52.541839,13.390056
4,4,-1,-1,"POLYGON ((1494611.363 6896995.788, 1494708.149...",52.527898,13.444455
...,...,...,...,...,...,...
266,266,-9,-7,"POLYGON ((1522486.772 6893415.233, 1522578.827...",52.508285,13.694856
267,267,-9,-9,"POLYGON ((1526156.027 6900082.526, 1526247.513...",52.544718,13.727827
268,268,-7,-9,"POLYGON ((1521933.780 6905992.816, 1522026.023...",52.577001,13.689907
269,269,-8,-9,"POLYGON ((1524045.347 6903037.871, 1524137.212...",52.560864,13.708871


In [8]:


gdf_madrid= gdf_madrid.to_crs(epsg=3857)
gdf_berlin= gdf_berlin.to_crs(epsg=3857)

gdf_madrid_mapped_buildings = gpd.sjoin(gdf_madrid, madrid_hexagons_gdf).drop(columns=["index_right"])

gdf_berlin_mapped_buildings = gpd.sjoin(gdf_berlin, berlin_hexagons_gdf).drop(columns=["index_right"])

gdf_berlin_mapped_buildings


,building_id,height,age,type,id_source,type_source,geometry,hexagon_id,hexagon_x,hexagon_y,lat,lon
0,v0.1-DEU.3.1.1.1_1-0,2.77,NaN,NaN,BLDG_00030000006a7af7,2862.0,"POLYGON ((1510943.155 6878374.869, 1510943.157...",143,-7,-1,52.431146,13.558297
1,v0.1-DEU.3.1.1.1_1-1,2.38,NaN,NaN,BLDG_00030000006a7ac7,2862.0,"POLYGON ((1510930.698 6878361.523, 1510930.696...",143,-7,-1,52.431146,13.558297
2,v0.1-DEU.3.1.1.1_1-2,5.43,NaN,NaN,BLDG_0003000f00190176,1301.0,"POLYGON ((1510930.690 6878423.168, 1510930.689...",143,-7,-1,52.431146,13.558297
3,v0.1-DEU.3.1.1.1_1-3,4.27,NaN,NaN,BLDG_000300090023bdbd,1004.0,"POLYGON ((1510934.982 6878223.712, 1510934.981...",143,-7,-1,52.431146,13.558297
4,v0.1-DEU.3.1.1.1_1-4,3.31,NaN,NaN,BLDG_0003000f00190236,1301.0,"POLYGON ((1510813.713 6878448.665, 1510813.711...",143,-7,-1,52.431146,13.558297
...,...,...,...,...,...,...,...,...,...,...,...,...
538438,v0.1-DEU.3.1.1.1_1-538438,0.93,NaN,NaN,DEB_LOD2_UUID_2b05c943-0a28-4567-89c9-e652996d...,NaN,"POLYGON ((1485916.101 6910904.824, 1485916.100...",70,4,-1,52.608301,13.349368
538449,v0.1-DEU.3.1.1.1_1-538449,1.54,NaN,NaN,DEB_LOD2_UUID_c1ef21de-50ca-4d37-8118-33e4f456...,NaN,"POLYGON ((1486106.968 6910788.338, 1486106.966...",70,4,-1,52.608301,13.349368
538454,v0.1-DEU.3.1.1.1_1-538454,2.49,NaN,NaN,DEB_LOD2_UUID_c2fe762c-e122-40d9-9866-7102e0df...,NaN,"POLYGON ((1485910.027 6910886.942, 1485910.027...",70,4,-1,52.608301,13.349368
538460,v0.1-DEU.3.1.1.1_1-538460,2.88,NaN,NaN,DEB_LOD2_UUID_f73dce9d-8b92-4b65-89ee-ede0a978...,NaN,"POLYGON ((1484739.903 6909988.569, 1484739.903...",70,4,-1,52.608301,13.349368


In [9]:
gdf_madrid_mapped_buildings.to_pickle("./data/gdf_madrid_mapped_buildings.pkl")  
gdf_berlin_mapped_buildings.to_pickle("./data/gdf_berlin_mapped_buildings.pkl")  

In [10]:
gdf_list= [gdf_madrid_mapped_buildings, gdf_berlin_mapped_buildings]



In [11]:

from pyproj import Transformer


transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326") # transform geometry to lat/lon
for g in gdf_list:
    poly = g['geometry']
    g['lat'] ,  g['lon'] =  transformer.transform(poly.centroid.x,poly.centroid.y) 
    #print(transformer.transform(poly))
gdf_list[0]

,building_id,height,age,type,id_source,type_source,geometry,hexagon_id,hexagon_x,hexagon_y,lat,lon
111,v0.1-ESP.8.1.3.10_1-111,2.5,2000.0,non-residential,ES.SDGC.BU.000900200VK36F,3_industrial,"POLYGON ((-414530.441 4916181.526, -414527.672...",56,0,4,40.344900,-3.723747
1389,v0.1-ESP.8.1.3.10_1-1389,12.5,1966.0,residential,ES.SDGC.BU.0062501VK4606C,1_residential,"POLYGON ((-412665.414 4915920.501, -412665.738...",56,0,4,40.343081,-3.706977
1390,v0.1-ESP.8.1.3.10_1-1390,12.5,1964.0,residential,ES.SDGC.BU.0062502VK4606C,1_residential,"POLYGON ((-412627.139 4915948.269, -412624.368...",56,0,4,40.343204,-3.706670
1391,v0.1-ESP.8.1.3.10_1-1391,12.5,1964.0,residential,ES.SDGC.BU.0062503VK4606C,1_residential,"POLYGON ((-412606.092 4915955.096, -412603.857...",56,0,4,40.343256,-3.706467
1392,v0.1-ESP.8.1.3.10_1-1392,12.5,1966.0,residential,ES.SDGC.BU.0062504VK4606C,1_residential,"POLYGON ((-412558.450 4915955.293, -412558.597...",56,0,4,40.343297,-3.706136
...,...,...,...,...,...,...,...,...,...,...,...,...
155442,v0.1-ESP.8.1.3.10_1-155442,17.5,2016.0,residential,ES.SDGC.BU.4649302VK5744N,1_residential,"POLYGON ((-393691.940 4927458.854, -393690.537...",68,-5,-4,40.422089,-3.536451
155443,v0.1-ESP.8.1.3.10_1-155443,2.5,2008.0,residential,ES.SDGC.BU.4649901VK5744N,1_residential,"POLYGON ((-393635.354 4927405.639, -393633.267...",68,-5,-4,40.421653,-3.536062
155444,v0.1-ESP.8.1.3.10_1-155444,17.5,2008.0,residential,ES.SDGC.BU.4649901VK5744N_part_1,1_residential,"POLYGON ((-393653.135 4927436.465, -393651.311...",68,-5,-4,40.422019,-3.535711
155445,v0.1-ESP.8.1.3.10_1-155445,17.5,2008.0,residential,ES.SDGC.BU.4748301VK5744N,1_residential,"POLYGON ((-393516.381 4927411.612, -393515.924...",68,-5,-4,40.421582,-3.535083


import pyproj

from pyproj import Transformer
from shapely.ops import transform


project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:3857'), # source coordinate system
    pyproj.Proj(init='epsg:4326')) # destination coordinate system


for g in gdf_list:
    poly = g['geometry']
    print('hello1')
    print(poly)
    trans_g =  transform(project.transform, poly)
    print('hello2')
    print(trans_g)
    #print(transformer.transform(poly))
gdf_list[0]

In [12]:
entity_id_list = [ "urn:ngsi-ld:Building:building-Madrid", "urn:ngsi-ld:Building:building-Berlin"]
entity_type_list = ['Building'] * len(gdf_list)


In [13]:
import time
from geojson.geometry import Point, Polygon, MultiLineString

example_flag=False
for entity_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True

    hexagon_id_list = tmp_gdf[['hexagon_id']].drop_duplicates().values.tolist()
    print("Info: Unique hexagons for the dataset : "+ str(hexagon_id_list))
    feature_list= tmp_gdf.columns.tolist()
    dataset_length = len(tmp_gdf)
    for hexagon_id in hexagon_id_list:
        hexagon_id = hexagon_id[0]
        hexagon_gdf = tmp_gdf[tmp_gdf['hexagon_id']== hexagon_id]

        counter =0
        for index, row in hexagon_gdf.iterrows():
            lat=0
            lon=0
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type_list[entity_index], entity_id_list[entity_index] + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity
            for f in range(len(feature_list)):
                
                if feature_list[f] =='type': feature_list[f] ='building_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
                if feature_list[f] == 'lat': lat = row[f]
                if feature_list[f] == 'lon': lon=row[f]
                
                if lat !=0 and lon !=0: # both values are obtained for location, add the location attribute 
                    location_value = Point((lon, lat))
                    entity_obj.add_geoprop("location", location_value) # MAKE THIS A POINT (SIVA EXAMPLE)
                if(feature_list[f] not in ['datetime','type']):
                    entity_obj.add_prop(feature_list[f], str(row[f]))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(hexagon_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
        print("... published data for hexagon: " +  str(hexagon_id))

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully with entity ID: ' + str(entity_id_list[entity_index]) 
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))


Info: Unique hexagons for the dataset : [[56], [57], [73], [50], [60], [58], [11], [18], [77], [62], [26], [63], [30], [78], [13], [52], [44], [81], [43], [66], [80], [46], [6], [16], [67], [24], [5], [39], [53], [37], [21], [65], [41], [48], [42], [22], [69], [20], [23], [61], [14], [4], [71], [40], [70], [38], [64], [28], [0], [25], [79], [75], [12], [49], [3], [90], [76], [84], [45], [47], [89], [55], [15], [2], [29], [1], [7], [10], [87], [59], [9], [34], [36], [31], [35], [17], [19], [8], [32], [54], [27], [33], [85], [83], [68]]
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': 'urn:ngsi-ld:Building:building-Madrid:Hexagon:56', 'type': 'Building', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'building_id': {'type': 'Property', 'value': 'v0.1-ESP.8.1.3.10_1-111'}, 'height': {'type': 'Property', 'value': '2.5'}, 'age': {'type': 'Property', 'value': '2000.0'}, 'building_type': {'type': 'Property', 'value': '